# Pipeline

In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import setting
from sklearn import set_config

from src import viz
from src.common import util

In [3]:
sns.set_theme(style="whitegrid")

In [4]:
df_train, df_test = util.load_data()

In [11]:
X = df_train.drop(columns=["id", "target"])
y = df_train["target"]

In [29]:
from sklearn.ensemble import RandomTreesEmbedding, HistGradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.model_selection import check_cv, KFold
from sklearn.base import clone, BaseEstimator



In [18]:
model = HistGradientBoostingRegressor()

In [8]:
cv = 5
cv = check_cv(cv)
cv

KFold(n_splits=5, random_state=None, shuffle=False)

In [27]:
def oof_predict(
    estimator: BaseEstimator,
    X: pd.DataFrame,
    y: pd.Series,
    cv=None,
):
    # splitter
    splitter = check_cv(cv)

    # oof予測値のデータフレームの準備
    df_oof = pd.DataFrame()
    df_oof["target"] = y
    df_oof["oof"] = np.nan
    df_oof["fold"] = np.nan

    # モデルを格納するリスト
    models = []

    # ndarrayにする
    X_array = np.asarray(X)
    y_array = np.asarray(y)

    for num_fold, (train_index, val_index) in enumerate(splitter.split(X, y)):
        # model clone
        _model = clone(model)

        # train val の分割
        X_train, X_val = X_array[train_index], X_array[val_index]
        y_train, y_val = y_array[train_index], y_array[val_index]

        # fit
        _model.fit(X_train, y_train)

        # oof predict
        pred = _model.predict(X_val)
        df_oof.iloc[val_index, df_oof.columns.get_loc("oof")] = pred
        df_oof.iloc[val_index, df_oof.columns.get_loc("fold")] = num_fold

        # モデルをリストに格納
        models.append(_model)

    return df_oof, models

In [30]:
df_oof, models = oof_predict(
    HistGradientBoostingRegressor(),
    X,
    y,
    KFold(5, shuffle=True, random_state=0)
)

In [31]:
df_oof

,target,oof,fold
0,7.243043,7.853324,1.0
1,8.203331,7.889643,3.0
2,7.776091,7.567879,1.0
3,6.957716,7.741125,3.0
4,7.951046,7.865333,4.0
...,...,...,...
299995,7.385215,7.547144,2.0
299996,7.242617,7.922011,2.0
299997,9.592487,8.736977,2.0
299998,8.207951,8.428559,2.0


In [32]:
models

[HistGradientBoostingRegressor(),
 HistGradientBoostingRegressor(),
 HistGradientBoostingRegressor(),
 HistGradientBoostingRegressor(),
 HistGradientBoostingRegressor()]